In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sys
from ..src.test import *
%load_ext autoreload
%autoreload 2
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.options.mode.chained_assignment = None  # default='warn'
housing = pd.read_csv('data/nashville-housing.csv')

ImportError: attempted relative import with no known parent package

In [ ]:

#Filter out Land Use Categories with less than 10 entries
filtered_landuse = remove_small_groupings(housing,"LandUse",10)
filtered_landuse['LandUse'] = filtered_landuse['LandUse'].replace('VACANT RES LAND', 'VACANT RESIDENTIAL LAND')
t_housing = remove_unwanted_columns(filtered_landuse,['OwnerName','OwnerAddress','LegalReference'])
# t_housing = filtered_landuse.drop(columns=['OwnerName','OwnerAddress','LegalReference'])
t_housing = filtered_landuse.dropna(subset=['Acreage', 'TaxDistrict','LandValue','BuildingValue',"SalePrice"])
t_housing['SalePrice'] = pd.to_numeric(t_housing.loc[:,'SalePrice'],errors='coerce')
t_housing


In [ ]:

t_housing = remove_unwanted_columns(filtered_landuse,['OwnerName','OwnerAddress','LegalReference'])
# t_housing = filtered_landuse.drop(columns=['OwnerName','OwnerAddress','LegalReference'])
t_housing = filtered_landuse.dropna(subset=['Acreage', 'TaxDistrict','LandValue','BuildingValue',"SalePrice"])
t_housing['SalePrice'] = pd.to_numeric(t_housing.loc[:,'SalePrice'],errors='coerce')

In [ ]:
pie_group = t_housing['LandUse'].value_counts().head(5)

In [ ]:
pie_group.sort_values(ascending=False)

In [ ]:

pie_group.plot(kind='pie',title="Nashville Land Use Percentages", figsize=(20,9),ylabel=" ",  labels=pie_group.index, autopct ="%1.1f%%" )


In [ ]:
tax_district= t_housing.groupby('TaxDistrict')["LandUse"].size()
ax = tax_district.plot.barh(title="Total Properties Per Tax District", ylabel="")
ax.ticklabel_format(style='plain', axis='x')
for i, count in enumerate(tax_district):
    ax.annotate(str(count), xy=(count, i), va='center')


In [ ]:
tax_district= t_housing.groupby(["LandUse",'TaxDistrict']).size().sort_values(ascending=False).head(10)
tax_district= tax_district.sort_values(ascending=True)
ax = tax_district.plot.barh(title="Total Properties Per Tax District", ylabel="",figsize=(16,16))
ax.ticklabel_format(style='plain', axis='x')
for i, count in enumerate(tax_district):
    ax.annotate(str(count), xy=(count, i), va='center')

In [ ]:
tax_district= t_housing.groupby(["TaxDistrict","LandUse"])["SalePrice"].mean().to_frame()
tax_district=tax_district.sort_values(by=["TaxDistrict","SalePrice"], ascending=False)
fig, ax = plt.subplots(figsize=(15,15))
tax_district= filter_outliers(tax_district, 'SalePrice')
tax_district.plot(kind='barh', stacked=True,ax=ax)
ax.ticklabel_format(style='plain', axis='x')
ax.set_title("Price Averages Per Land Use Category")
ax.set_ylabel("Land Use Groups")
ax.set_xlabel("Sale Price")




In [ ]:
t_housing['AcreageToCost'] = t_housing['LandValue']/ t_housing["Acreage"]
t_housing['SaleDifference'] = t_housing['SalePrice'] - t_housing["TotalValue"]
average_cost = t_housing.groupby(["LandUse"]).mean().sort_values(by = "SaleDifference", ascending=False)
average_cost = average_cost.drop("UniqueID", axis=1)
average_cost = average_cost.fillna(0)
average_cost


In [ ]:
average_cost_chart = average_cost[["LandValue","SalePrice","BuildingValue","TotalValue","SaleDifference"]].sort_values(by='TotalValue',ascending=True)
average_cost_chart = average_cost_chart[average_cost_chart['TotalValue'] != 0]
fig, ax = plt.subplots(figsize=(15,15))

# Plotting the Total Value as a line plot
average_cost_chart['TotalValue'].plot(kind="barh",ax=ax, color='red')
# Plotting the mean values of Land Value and Building Value as horizontal stacked bars
average_cost_chart[['LandValue', 'BuildingValue']].plot(kind='barh', stacked=True, ax=ax, color=['green','blue'])
ax.legend(["Total Value","Land Value", "Building Value"])
plt.xlabel('Mean Cost')
plt.ylabel('Land Use')
plt.title('Mean Cost Breakdown of Land by Categories')



In [ ]:

th_filtered = filter_outliers(t_housing, 'SaleDifference')
ax =th_filtered.boxplot("SaleDifference", 
                  by="LandUse", 
                figsize=(16,16), 
                showfliers = False, 
                vert = False, patch_artist=True, 
                boxprops = dict(facecolor = "grey"),
                medianprops = dict(color = "red", linewidth = 1.5),
                whiskerprops = dict(linewidth = 2),
                capprops = dict(linewidth = 2))
ax.ticklabel_format(style='plain', axis='x')
ax.set_title("Price Ranges Per Land Use Category")
ax.set_ylabel("Land Use Groups")
ax.set_xlabel("Sale Price")